# (Neo4j)-[:LOVES]->(Kafka)

<div class="img-responsive center-block" style="background-image: url('https://cdn-images-1.medium.com/max/2000/1*0k7QFFBl7YGD9haNYeLQpw.png'); width: 1124px; height: 300px; background-position: center; background-size: cover;"></div>

## Link

https://neo4j-contrib.github.io/neo4j-streams/

# Initialize spark & neo4j sessions

In [ ]:
# Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').config(
    "spark.jars.packages",
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2"
).getOrCreate()

# Neo4j
import sys
!{sys.executable} -m pip install py2neo

from py2neo import Graph

graph = Graph("bolt://neo4j:7687", auth=("neo4j", "streams"))

# Query the JIT-DWH

In [ ]:
from pyspark.sql.functions import first

flattenedDF = spark.read.format("json").load("/home/streams/jit-dwh/**") \
    .where("neo_id is not null") \
    .groupBy("neo_id", "timestamp", "host", "labels", "operation") \
    .pivot("key") \
    .agg(first("value"))

flattenedDF.show()

# Let's inspect the Structure of our data

In [ ]:
flattenedDF.printSchema()

# Let's go to add the field "placeOfBirth" over a node

In [ ]:
id = input("Id: ")
placeOfBirth = input("Place of Birth: ")

graph.run("MATCH (p:Person{id: $id}) SET p.placeOfBirth = $placeOfBirth return p", parameters = {"id": id, "placeOfBirth": placeOfBirth}).data()

# Check how the new field is automatically added to our representation

In [ ]:
from pyspark.sql.functions import first, desc

flattenedDF = spark.read.format("json").load("/home/streams/jit-dwh/**") \
    .where("neo_id is not null") \
    .groupBy("neo_id", "timestamp", "host", "labels", "operation") \
    .pivot("key") \
    .agg(first("value"))

id = input("Id: ")
flattenedDF.where("id = '{id}'".format(**locals())).orderBy(desc("timestamp")).show()